In [8]:
import pandas as pd

df_iea = pd.read_csv('../data/processed/iea_wide_format.csv')
df_stations = pd.read_csv('../data/processed/stations_enhanced.csv')

In [9]:
station_stats = df_stations.groupby('country').agg({
    'Station ID': 'count',
    'Cost (USD/kWh)': 'mean',
    'Reviews (Rating)': 'mean',
    'is_fast_charger': 'mean',
    'is_renewable': 'mean',
    'is_24_7': 'mean',
    'Usage Stats (avg users/day)': 'mean'
}).rename(columns={
    'Station ID': 'total_stations',
    'Cost (USD/kWh)': 'avg_cost_per_kwh',
    'Reviews (Rating)': 'avg_rating',
    'is_fast_charger': 'fast_charger_ratio',
    'is_renewable': 'renewable_ratio',
    'is_24_7': 'always_available_ratio',
    'Usage Stats (avg users/day)': 'avg_daily_users'
})

station_stats.head()

,total_stations,avg_cost_per_kwh,avg_rating,fast_charger_ratio,renewable_ratio,always_available_ratio,avg_daily_users
country,,,,,,,
Australia,295,0.306576,4.042373,0.301695,0.515254,0.308475,55.559322
Brazil,279,0.290609,3.992832,0.322581,0.505376,0.383513,52.311828
Canada,257,0.296070,4.021012,0.346304,0.470817,0.322957,56.719844
China,319,0.306050,4.016928,0.288401,0.514107,0.332288,56.692790
Germany,292,0.304897,3.947603,0.339041,0.472603,0.359589,55.527397


In [10]:
df_merged = df_iea.merge(
    station_stats,
    left_on='region',
    right_index=True,
    how='left'
)

df_merged.head()

,region,year,category,mode,powertrain,ev_charging_points,ev_sales,ev_sales_share,ev_stock,ev_stock_share,electricity_demand,oil_displacement_mbd,"oil_displacement,_million_lge",total_stations,avg_cost_per_kwh,avg_rating,fast_charger_ratio,renewable_ratio,always_available_ratio,avg_daily_users
0,Australia,2011,Historical,Cars,BEV,NaN,49.0,NaN,49.0,NaN,NaN,NaN,NaN,295.0,0.306576,4.042373,0.301695,0.515254,0.308475,55.559322
1,Australia,2011,Historical,Cars,EV,NaN,NaN,0.0065,NaN,0.00039,NaN,NaN,NaN,295.0,0.306576,4.042373,0.301695,0.515254,0.308475,55.559322
2,Australia,2012,Historical,Cars,BEV,NaN,170.0,NaN,220.0,NaN,NaN,NaN,NaN,295.0,0.306576,4.042373,0.301695,0.515254,0.308475,55.559322
3,Australia,2012,Historical,Cars,EV,NaN,NaN,0.0300,NaN,0.00240,NaN,NaN,NaN,295.0,0.306576,4.042373,0.301695,0.515254,0.308475,55.559322
4,Australia,2012,Historical,Cars,PHEV,NaN,80.0,NaN,80.0,NaN,NaN,NaN,NaN,295.0,0.306576,4.042373,0.301695,0.515254,0.308475,55.559322


In [11]:
df_merged['stations_per_million_evs'] = (
    df_merged['total_stations'] / (df_merged['ev_stock'] / 1_000_000)
)

In [12]:
df_merged = df_merged[df_merged['category'] == 'Historical'].copy()

df_merged.head()

,region,year,category,mode,powertrain,ev_charging_points,ev_sales,ev_sales_share,ev_stock,ev_stock_share,...,oil_displacement_mbd,"oil_displacement,_million_lge",total_stations,avg_cost_per_kwh,avg_rating,fast_charger_ratio,renewable_ratio,always_available_ratio,avg_daily_users,stations_per_million_evs
0,Australia,2011,Historical,Cars,BEV,NaN,49.0,NaN,49.0,NaN,...,NaN,NaN,295.0,0.306576,4.042373,0.301695,0.515254,0.308475,55.559322,6.020408e+06
1,Australia,2011,Historical,Cars,EV,NaN,NaN,0.0065,NaN,0.00039,...,NaN,NaN,295.0,0.306576,4.042373,0.301695,0.515254,0.308475,55.559322,NaN
2,Australia,2012,Historical,Cars,BEV,NaN,170.0,NaN,220.0,NaN,...,NaN,NaN,295.0,0.306576,4.042373,0.301695,0.515254,0.308475,55.559322,1.340909e+06
3,Australia,2012,Historical,Cars,EV,NaN,NaN,0.0300,NaN,0.00240,...,NaN,NaN,295.0,0.306576,4.042373,0.301695,0.515254,0.308475,55.559322,NaN
4,Australia,2012,Historical,Cars,PHEV,NaN,80.0,NaN,80.0,NaN,...,NaN,NaN,295.0,0.306576,4.042373,0.301695,0.515254,0.308475,55.559322,3.687500e+06


In [13]:
df_merged.to_csv('../data/processed/merged_dataset.csv', index=False)
print(f"Saved: {len(df_merged)} rows, {len(df_merged.columns)} columns")

Saved: 5085 rows, 21 columns
